In [1]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

<h4><i>Axing tedious code</i></h4>

In [ ]:
data Query = Query
data SomeObj = SomeObj
data IoOnlyObj = IoOnlyObj
data Err = Err

In [ ]:
decodeFn :: String -> Either Err SomeObj
decodeFn = undefined

fetchFn :: Query -> IO [String]
fetchFn = undefined

makeIoOnlyObj :: [SomeObj] -> IO [(SomeObj, IoOnlyObj)]
makeIoOnlyObj = undefined

In [ ]:
pipelineQuery :: Query -> IO (Either Err [(SomeObj, IoOnlyObj)])
pipelineQuery query = do
    a <- fetchFn query
    case sequence (map decodeFn a) of
        (Left err) -> return $ Left err
        (Right res) -> do
            a <- makeIoOnlyObj res
            return $ Right a

In [ ]:
pipelineQuery' :: Query -> IO (Either Err [(SomeObj, IoOnlyObj)])
pipelineQuery' = ((traverse makeIoOnlyObj . traverse decodeFn) =<<) . fetchFn

<h4>Http Clients</h4>

In [ ]:
import Data.ByteString.Lazy hiding (map)
import Network.Wreq

In [ ]:
urls :: [String]
urls = [ "http://httpbin.org/ip"
        ,"http://httpbin.org/bytes/5"]

In [ ]:
mappingGet :: [IO (Response ByteString)]
mappingGet = map get urls

In [ ]:
traversedUrls :: IO [Response ByteString]
traversedUrls = traverse get urls

<h4>Exercises</h4>

In [ ]:
import Data.Functor.Identity
import Data.Functor.Constant
import Data.Monoid

In [ ]:
traverse (Identity . (+1)) [1..3]

In [ ]:
xs' = [1..5]
xs = fmap Sum xs'
traverse (Constant . (+1)) xs

<h4>Either synonym</h4>

In [ ]:
data Sum a b = First a | Second b deriving (Eq, Ord, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Sum a b) where
    arbitrary = frequency [(1, First <$> arbitrary), (3, Second <$> arbitrary)]
    
instance (Eq a, Eq b) => EqProp (Sum a b) where
    (=-=) = eq

instance Functor (Sum a) where
    fmap _ (First x) = First x
    fmap f (Second y) = Second (f y)
    
instance Applicative (Sum a) where
    pure = Second
    First e <*> _ = First e
    _ <*> First x = First x
    Second f <*> y = fmap f y

In [ ]:
instance Foldable (Sum a) where
    foldMap _ (First _) = mempty
    foldMap f (Second y) = f y
    
    foldr _ z (First _) = z
    foldr f z (Second y) = f y z
    
instance Traversable (Sum a) where
    traverse _ (First x) = pure (First x)
    traverse f (Second y) = Second <$> f y

In [ ]:
trigger :: Sum (String, String, String) (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (applicative trigger)
quickBatch (traversable trigger)

<h4>Tuple Synonym</h4>

In [ ]:
data Tupley a b = Tupley {first:: a, second:: b} deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Tupley a b) where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        return (Tupley a b)
        
instance (Eq a, Eq b) => EqProp (Tupley a b) where
    (=-=) = eq

In [ ]:
instance Functor (Tupley a) where
    fmap f (Tupley x y) = Tupley x (f y)
    
instance Monoid a => Applicative (Tupley a) where
    pure x = Tupley mempty x
    (<*>) (Tupley fx fy) (Tupley x y) = Tupley (fx <> x) (fy y)

In [ ]:
instance Foldable (Tupley a) where
    foldMap f (Tupley a b) = f b
    foldr f acc (Tupley a b) = f b acc
    
instance Traversable (Tupley a) where
    traverse f (Tupley x y) = (Tupley x) <$> f y

In [ ]:
trigger :: Tupley (String, String, String) (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (applicative trigger)
quickBatch (traversable trigger)

<h4>Identity</h4>

In [ ]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Arbitrary a => Arbitrary (Identity a) where
    arbitrary = Identity <$> arbitrary
    
instance Eq a => EqProp (Identity a) where
    (=-=) = eq

In [ ]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)
    
instance Foldable Identity where
    foldMap f (Identity a) = f a
    foldr f b (Identity a) = f a b
    
instance Traversable Identity where
    traverse f (Identity a) = Identity <$> f a

In [ ]:
trigger :: Identity (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (traversable trigger)

<h4>Listy</h4>

In [ ]:
data Listy a = Nil | Cons {head':: a, tail':: Listy a} deriving (Eq, Show)

instance Arbitrary a => Arbitrary (Listy a) where
    arbitrary = frequency [(1, pure Nil), (5, Cons <$> arbitrary <*> arbitrary)]
    
instance Eq a => EqProp (Listy a) where
    (=-=) = eq

In [ ]:
instance Functor Listy where
    fmap _ Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)
    
instance Foldable Listy where
    foldMap _ Nil = mempty
    foldMap f (Cons x xs) = f x <> foldMap f xs
    
    foldr _ b Nil = b
    foldr f b (Cons x xs) = f x (foldr f b xs)
    
instance Traversable Listy where
    traverse _ Nil = pure Nil
    traverse f (Cons x xs) = Cons <$> f x <*> traverse f xs

In [ ]:
trigger :: Listy (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (traversable trigger)

<h4>Big and Bigger</h4>

In [ ]:
data Big a b = Big a b b deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Big a b) where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        return (Big a b b)
        
instance (Eq a, Eq b) => EqProp (Big a b) where
    (=-=) = eq

In [ ]:
instance Functor (Big a) where
    fmap f (Big a b b') = Big a (f b) (f b')
    
instance Foldable (Big a) where
    foldMap f (Big a b b') = (f b) <> (f b')
    foldr f acc (Big a b b') = f b (f b' acc)
    
instance Traversable (Big a) where
    traverse f (Big a b b') = Big a <$> f b <*> f b'

In [ ]:
trigger :: Big String (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (traversable trigger)

<h4>S</h4>

In [ ]:
data S n a = S (n a) a deriving (Eq, Show)

instance (Functor n, Arbitrary (n a), Arbitrary a) => Arbitrary (S n a) where
    arbitrary = S <$> arbitrary <*> arbitrary
    
instance (Applicative n, Testable (n Property), EqProp a) => EqProp (S n a) where
    (S x y) =-= (S p q) = (property $ (=-=) <$> x <*> p) .&. (y =-= q)

In [ ]:
instance Functor n => Functor (S n) where
    fmap f (S x y) = S (fmap f x) (f y)

In [ ]:
functorIdentity :: (Functor f, Eq (f a)) => f a -> Bool
functorIdentity f = fmap id f == f

functorCompose :: (Functor f, Eq (f c)) =>
                    (a -> b) -> (b -> c) -> f a -> Bool
functorCompose f g x = fmap g (fmap f x) == fmap (g.f) x

In [ ]:
f :: S [] String -> Bool
f = functorIdentity

quickCheck f

c = functorCompose (++ "HI") ((++) "Bye")
li x = c (x :: S [] String)

quickCheck li

In [ ]:
instance Foldable n => Foldable (S n) where
    foldMap f (S x y) = f y <> foldMap f x 

In [ ]:
instance Traversable n => Traversable (S n) where
    traverse f (S x y) = S <$> traverse f x <*> f y

<h4>Tree</h4>

In [2]:
data Tree a = Empty | Leaf a | Node (Tree a) a (Tree a) deriving (Eq, Show)

In [3]:
instance Arbitrary a => Arbitrary (Tree a) where
  arbitrary = genTree

genTree :: Arbitrary a => Gen (Tree a)
genTree = do
  x <- arbitrary
  n <- genTree
  n' <- genTree
  frequency [ (1, return Empty)
            , (2, return $ Leaf x)
            , (2, return $ Node n x n') ]
    
instance Eq a => EqProp (Tree a) where
    (=-=) = eq

In [4]:
instance Functor Tree where
    fmap _ Empty = Empty
    fmap f (Leaf a) = Leaf (f a)
    fmap f (Node lc a rc) = Node (f <$> lc) (f a) (f <$> rc)

In [12]:
instance Foldable Tree where
    foldMap f Empty = mempty
    foldMap f (Leaf a) = f a
    foldMap f (Node lc a rc) = foldMap f lc <> f a <> foldMap f rc
    
    foldr f b Empty = b
    foldr f b (Leaf a) = f a b
    foldr f b (Node lc a rc) = f a (foldr f (foldr f b rc) lc)

In [15]:
instance Traversable Tree where
    traverse f Empty = pure Empty
    traverse f (Leaf a) = Leaf <$> f a
    traverse f (Node lc a rc) = Node <$> traverse f lc <*> f a <*> traverse f rc

In [16]:
trigger :: Tree (String, String, String)
trigger = undefined

quickBatch (functor trigger)
quickBatch (traversable trigger)


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

In [13]:
:t traverse

traverse :: forall (t :: * -> *) (f :: * -> *) a b. (Traversable t, Applicative f) => (a -> f b) -> t a -> f (t b)

In [14]:
traverse Just [1, 2, 3]

Just [1,2,3]